In [2]:
import cv2
import numpy as np
import pyrealsense2 as rs

# Initialize the RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)  # Enable depth stream
pipeline.start(config)

# Create a list to store trackers and tracked data
trackers = []
tracked_data = []

while True:
    # Wait for the next set of frames
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    depth_frame = frames.get_depth_frame()  # Retrieve depth frame
    
    if color_frame and depth_frame:
        # Convert the color frame to a numpy array
        color_image = np.asanyarray(color_frame.get_data())
        
        if len(trackers) == 0:
            # Display instructions
            cv2.putText(color_image, "Select object(s) to track and press 's' to start tracking", (20, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        for tracker in trackers:
            # Update each tracker
            ok, bbox = tracker.update(color_image)
            
            if ok:
                # Tracking successful, draw bounding box
                p1 = (int(bbox[0]), int(bbox[1]))
                p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                cv2.rectangle(color_image, p1, p2, (0, 255, 0), 2)
                
                # Retrieve depth value at the center of the bounding box
                center_x = int((p1[0] + p2[0]) / 2)
                center_y = int((p1[1] + p2[1]) / 2)
                depth_value = depth_frame.get_distance(center_x, center_y)
                
                # Store the bounding box coordinates and depth value
                tracked_data.append(p1 + p2 + (depth_value,))
            else:
                # Tracking failed, remove tracker
                trackers.remove(tracker)

        # Display the color image
        cv2.imshow('Object Tracking', color_image)
        
    # Check for key press
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('s'):
        # Select objects to track when 's' is pressed
        bbox = cv2.selectROIs('Object Tracking', color_image, fromCenter=False)
        for box in bbox:
            tracker = cv2.TrackerCSRT_create()
            tracker.init(color_image, tuple(box))
            trackers.append(tracker)
    elif key == 27:
        # Exit program when 'ESC' is pressed
        break

# Stop the pipeline and close all OpenCV windows
pipeline.stop()
cv2.destroyAllWindows()

# Save the tracked data to a file
np.savetxt('tracked_data.txt', tracked_data)
